In [ ]:
import os, sys

p = "/kaggle/input/sage-zrok-token/.zrok_api_key"
zrok_token = None

if os.path.isfile(p):
    with open(p, "r", encoding="utf-8", errors="ignore") as f:
        zrok_token = f.read().strip()

if not zrok_token:
    print("❌ Token not found or empty:", p)
    sys.exit(1)

In [ ]:
import os
import shutil

print("Setting up models...")

# --- Copy Comi-Noir2 if not already present ---
source = "/kaggle/input/comi-noir2"
dest = "/kaggle/working/GraydientPlatformAPI/comi-noir2"

if os.path.exists(dest):
    print(f"✓ Comi-Noir2 already exists at {dest}, skipping copy")
else:
    print(f"  Copying Comi-Noir2...")
    shutil.copytree(source, dest)
    print(f"  ✓ Copied to {dest}")

# --- Copy ControlNet OpenPose if not already present ---
controlnet_source = "/kaggle/input/controlnet-openpose"
controlnet_dest = "/kaggle/working/sd-controlnet-openpose"

if os.path.exists(controlnet_dest):
    print(f"✓ ControlNet already exists at {controlnet_dest}, skipping copy")
else:
    print(f"  Copying ControlNet OpenPose...")
    shutil.copytree(controlnet_source, controlnet_dest)
    print(f"  ✓ Copied to {controlnet_dest}")

print("✅ All models ready!")

In [ ]:
# -----------------
# TXT2IMG pipeline
# -----------------

import os
from datetime import datetime
import torch
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, DPMSolverMultistepScheduler
from PIL import Image
from zipfile import ZipFile

# --- Device ---
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Model paths ---
MODEL_ID = "GraydientPlatformAPI/comi-noir2"
CONTROLNET_ID = "lllyasviel/sd-controlnet-openpose"
local_model_path = "/kaggle/working/GraydientPlatformAPI/comi-noir2"
local_controlnet_path = "/kaggle/working/sd-controlnet-openpose"

# --- Load or download ControlNet ---
if os.path.exists(local_controlnet_path):
    controlnet = ControlNetModel.from_pretrained(local_controlnet_path, torch_dtype=torch.float16).to(device)
else:
    controlnet = ControlNetModel.from_pretrained(CONTROLNET_ID, torch_dtype=torch.float16).to(device)
    controlnet.save_pretrained(local_controlnet_path)

# --- Load or download Stable Diffusion pipeline ---
if os.path.exists(local_model_path):
    pipe_txt2img = StableDiffusionControlNetPipeline.from_pretrained(local_model_path, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None).to(device)
else:
    pipe_txt2img = StableDiffusionControlNetPipeline.from_pretrained(MODEL_ID, controlnet=controlnet, torch_dtype=torch.float16, safety_checker=None).to(device)
    pipe_txt2img.save_pretrained(local_model_path)

# --- Fast scheduler ---
#pipe_txt2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_txt2img.scheduler.config)

from diffusers import EulerAncestralDiscreteScheduler
pipe_txt2img.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_txt2img.scheduler.config)

In [ ]:


# -------------------------------------
# TXT2IMG method [handles optional CN]
# -------------------------------------

from PIL import Image
import torch
from typing import Optional
import random

def _round_to_multiple(x: int, base: int = 8) -> int:
    return ((x + base - 1) // base) * base

def generate_txt2img(
    pipe_txt2img,
    prompt: str,
    negative_prompt: Optional[str] = None,
    height: int = 512,
    width: int = 512,
    num_inference_steps: int = 35,
    guidance_scale: float = 7.5,
    seed: Optional[int] = None,
    controlnet_image: Optional[Image.Image] = None,
    controlnet_scale: float = 1.0,
    use_controlnet: bool = False
):
    """
    Generate an image from text using a Stable Diffusion pipeline.
    Can optionally use ControlNet with a conditioning image.
    Returns a PIL.Image.
    """
    print("Starting TXT2IMG generation...")

    # Round dims to a multiple of 8 (stable-diffusion-friendly)
    height = _round_to_multiple(height, 8)
    width  = _round_to_multiple(width, 8)

    # Resize ControlNet conditioning image if provided
    if use_controlnet and controlnet_image is not None:
        controlnet_image = controlnet_image.resize((width, height))

    # Always pass an image to avoid pipeline errors
    pipeline_image = controlnet_image if use_controlnet else Image.new("RGB", (width, height), (0, 0, 0))

    # Create generator for reproducibility (if seed provided)
    generator = None

    # Generate random seed if none provided
    if seed is None:
        seed = random.randint(0, 2**32 - 1)

    generator = torch.Generator(device="cuda" if torch.cuda.is_available() else "cpu")
    generator.manual_seed(seed)

    # Build kwargs for the pipeline call
    kwargs = {
        "prompt": prompt,
        "negative_prompt": negative_prompt,
        "height": height,
        "width": width,
        "num_inference_steps": num_inference_steps,
        "guidance_scale": guidance_scale,
        "image": pipeline_image,
        "controlnet_conditioning_scale": controlnet_scale if use_controlnet else 0.0,
    }

    if generator is not None:
        kwargs["generator"] = generator

    # Run generation with appropriate context managers
    if torch.cuda.is_available():
        with torch.inference_mode(), torch.autocast("cuda"):
            output = pipe_txt2img(**kwargs)
    else:
        with torch.inference_mode():
            output = pipe_txt2img(**kwargs)

    print("TXT2IMG generation completed")
    return output.images[0]

In [ ]:



##### NU COMBINED PIPE IMG2IMG+CN


# ----------------------
# IMG2IMG + CN pipeline
# ----------------------

import os
import torch
from diffusers import (
    StableDiffusionImg2ImgPipeline,
    StableDiffusionControlNetImg2ImgPipeline,
    ControlNetModel,
    DPMSolverMultistepScheduler
)

# --- Device ---
device = "cuda" if torch.cuda.is_available() else "cpu"

# --- Model paths ---
MODEL_ID = "GraydientPlatformAPI/comi-noir2"
CONTROLNET_ID = "lllyasviel/sd-controlnet-openpose"
local_model_path = "/kaggle/working/GraydientPlatformAPI/comi-noir2"
local_controlnet_path = "/kaggle/working/sd-controlnet-openpose"


# ------------------ #
# --- ControlNet --- #
# ------------------ #
# --- Load or download ControlNet ---
if os.path.exists(local_controlnet_path):
    controlnet = ControlNetModel.from_pretrained(local_controlnet_path, torch_dtype=torch.float16).to(device)
else:
    controlnet = ControlNetModel.from_pretrained(CONTROLNET_ID, torch_dtype=torch.float16).to(device)
    controlnet.save_pretrained(local_controlnet_path)


# ------------------------ #
# --- Img2Img pipeline --- #
# ------------------------ #
if os.path.exists(local_model_path):
    pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
        local_model_path,
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)
else:
    pipe_img2img = StableDiffusionImg2ImgPipeline.from_pretrained(
        MODEL_ID,
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)
    pipe_img2img.save_pretrained(local_model_path)

# --- Fast scheduler ---
#pipe_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_img2img.scheduler.config)


from diffusers import EulerAncestralDiscreteScheduler
pipe_img2img.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_img2img.scheduler.config)

# --------------------------- #
# --- Img2Img CN pipeline --- #
# --------------------------- #
if os.path.exists(local_model_path):
    pipe_cn_img2img = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
        local_model_path,
        controlnet=controlnet,
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)
else:
    pipe_cn_img2img = StableDiffusionControlNetImg2ImgPipeline.from_pretrained(
        MODEL_ID,
        controlnet=controlnet,
        torch_dtype=torch.float16,
        safety_checker=None
    ).to(device)
    pipe_cn_img2img.save_pretrained(local_model_path)

# --- Fast scheduler ---
#pipe_cn_img2img.scheduler = DPMSolverMultistepScheduler.from_config(pipe_cn_img2img.scheduler.config)

from diffusers import EulerAncestralDiscreteScheduler
pipe_cn_img2img.scheduler = EulerAncestralDiscreteScheduler.from_config(pipe_cn_img2img.scheduler.config)


# TODO: test and reimplement:
#pipe_img2img.safety_checker = lambda images, **kwargs: (images, [False] * len(images))
#pipe_cn_img2img.safety_checker = lambda images, **kwargs: (images, [False] * len(images))

In [ ]:

# -------------------------------------
# img2img method [handles optional CN]
# -------------------------------------

import torch
import random

def generate_img2img(
    pipe_img2img,
    pipe_cn_img2img,
    prompt,
    negative_prompt,
    init_image,
    control_image=None,
    strength=0.8,
    guidance_scale=8.0,
    controlnet_conditioning_scale=1.0,
    num_inference_steps=50,
    seed=None,
    dtype=torch.float16,
    device="cuda",
):
    """
    Unified Img2Img function:
    - if control_image is None -> use plain Img2Img pipeline
    - if control_image is provided -> use ControlNet Img2Img pipeline
    """

    if control_image is not None:
        if control_image.size != init_image.size:
            control_image = control_image.resize(init_image.size)

    # Generate random seed if none provided
    if seed is None:
        seed = random.randint(0, 2**32 - 1)

    # Create generator
    generator = torch.Generator(device=device).manual_seed(seed)

    # Ensure proper autocast context
    with torch.inference_mode(), torch.autocast("cuda", dtype=dtype):
        if control_image is None:
            # Plain Img2Img
            output = pipe_img2img(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=init_image,
                strength=strength,
                guidance_scale=guidance_scale,
                num_inference_steps=num_inference_steps,
                generator=generator,
            )
        else:
            # ControlNet Img2Img
            output = pipe_cn_img2img(
                prompt=prompt,
                negative_prompt=negative_prompt,
                image=init_image,
                control_image=control_image,
                strength=strength,
                guidance_scale=guidance_scale,
                controlnet_conditioning_scale=controlnet_conditioning_scale,
                num_inference_steps=num_inference_steps,
                generator=generator,
            )

    return output

In [ ]:
# --------------------------
# FastAPI app init
# --------------------------

from fastapi import FastAPI
import nest_asyncio
import uvicorn

# Assume generate_txt2img, generate_img2img, pipe_txt2img, pipe_img2img are already defined
app = FastAPI()
nest_asyncio.apply()  # allow running uvicorn in Colab
#print("FastAPI app initialized.")

In [ ]:
# ---------------------------
# NEW TESTED T2I/I2I ENDPOINT /generate
# ---------------------------

from fastapi import FastAPI, Form, UploadFile, File
from fastapi.responses import StreamingResponse, JSONResponse
from io import BytesIO
from PIL import Image

img2img_inference_steps = 65

app = FastAPI()

def _parse_bool(val) -> bool:
    if val is None:
        return False
    return str(val).strip().lower() in ("1", "true", "yes", "y", "on")

@app.post("/generate")
async def generate(
    prompt: str = Form(...),
    negative_prompt: str = Form(None),
    height: int = Form(768),
    width: int = Form(768),
    num_inference_steps: int = Form(65),
    guidance_scale: float = Form(10.0),
    seed: int = Form(None),

    # img2img-only params
    strength: float = Form(0.7),
    init_image: UploadFile = File(None),  # optional, if present = img2img

    # controlnet
    use_controlnet: str = Form("0"),
    controlnet_scale: float = Form(1.0),
    control_image: UploadFile = File(None),  # optional
):
    """Unified endpoint: txt2img if no init_image, img2img if init_image provided."""

    # --- Load ControlNet image if present ---
    controlnet_img = None
    if control_image is not None:
        try:
            controlnet_img = Image.open(
                BytesIO(await control_image.read())
            ).convert("RGB")
        except Exception as e:
            return JSONResponse(
                status_code=400,
                content={"error": f"Failed to read control_image: {str(e)}"},
            )

    use_controlnet_flag = _parse_bool(use_controlnet) or controlnet_img is not None

    # --- Img2img path ---
    if init_image is not None:
        try:
            init_img = Image.open(BytesIO(await init_image.read())).convert("RGB")
        except Exception as e:
            return JSONResponse(
                status_code=400,
                content={"error": f"Failed to read init_image: {str(e)}"},
            )

        try:











            # -----------------------------
            # Run unified Img2Img method
            # -----------------------------



            generated_images = generate_img2img(
                pipe_img2img=pipe_img2img,
                pipe_cn_img2img=pipe_cn_img2img,
                prompt=prompt,
                negative_prompt=negative_prompt,
                init_image=init_img,
                control_image=controlnet_img,
                strength=strength,
                guidance_scale=guidance_scale,
                controlnet_conditioning_scale=1.8,
                num_inference_steps=num_inference_steps,
                seed=seed
            )

            generated_image = generated_images.images[0]



                #height=height,
                #width=width,
                #num_inference_steps=num_inference_steps,

                #seed=seed,

                #controlnet_scale=controlnet_scale,
                #use_controlnet=use_controlnet_flag,



        except Exception as e:
            return JSONResponse(status_code=500, content={"error": str(e)})

    # --- Txt2img path ---
    else:
        try:
            generated_image = generate_txt2img(
                pipe_txt2img=pipe_txt2img,
                prompt=prompt,
                negative_prompt=negative_prompt,
                height=height,
                width=width,
                num_inference_steps=num_inference_steps,
                guidance_scale=guidance_scale,
                seed=seed,
                controlnet_image=controlnet_img,
                controlnet_scale=controlnet_scale,
                use_controlnet=use_controlnet_flag,
            )
        except Exception as e:
            return JSONResponse(status_code=500, content={"error": str(e)})

    # --- Return PNG stream ---
    buffer = BytesIO()
    generated_image.save(buffer, format="PNG")
    buffer.seek(0)
    return StreamingResponse(buffer, media_type="image/png")

In [ ]:
# Download zrok v1.1.3 (latest)
!wget https://github.com/openziti/zrok/releases/download/v1.1.3/zrok_1.1.3_linux_amd64.tar.gz
!tar -xzf zrok_1.1.3_linux_amd64.tar.gz
!chmod +x zrok

In [ ]:
# Enable (automatic migration from 0.4)
!./zrok enable --headless "$zrok_token"

# Use the agent for better process management
#!./zrok agent start &
#!./zrok share public localhost:8000 --headless

In [ ]:
#!./zrok disable

In [ ]:
import uvicorn
import threading

def run_uvicorn():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

# Start in background thread
threading.Thread(target=run_uvicorn, daemon=True).start()

In [ ]:
import subprocess
import re
import time

def start_zrok_tunnel(port=8000):
    # Start the tunnel
    process = subprocess.Popen([
        "./zrok", "share", "public", f"localhost:{port}", "--headless"
    ], stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)

    # Give it a moment to start
    time.sleep(3)

    # Check agent status to get the URL
    status_process = subprocess.run([
        "./zrok", "agent", "status"
    ], capture_output=True, text=True)

    print("Agent Status:")
    print(status_process.stdout)

    return process

# Start the tunnel
tunnel_process = start_zrok_tunnel(8000)
print("Zrok tunnel started! Check the agent status above for your public URL.")

In [ ]:
!./zrok overview

In [ ]:
import time

print("Server and zrok tunnel are running. Keeping the notebook alive...")

try:
    while True:
        time.sleep(60)
except KeyboardInterrupt:
    print("Shutting down.")